In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [2]:
URL = 'https://m.imdb.com/chart/toptv/'

In [3]:
Film_title_list = []
rating_list=[]
Release_year_list = []

Director_list = []
Stars_list = []


headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
page = requests.get(URL, headers = headers)
soup = BeautifulSoup(page.text , 'html.parser')

# Film Title
Film_titles = soup.findAll('div' ,{'class':"ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN cli-title"})
for Film_title in Film_titles:
  Film_title_list.append(Film_title.contents[0].find("h3").text)

# Rating
ratings = soup.findAll('svg' ,{'class':"ipc-icon ipc-icon--star-inline"})
for rating in ratings:
  rating_list.append(rating.find_next(text=True).strip())

# Release Year
release_years = soup.findAll('div' ,{'class':"sc-b189961a-7 feoqjK cli-title-metadata"})
for release_year in release_years:
  Release_year_list.append(release_year.contents[0].text.strip())

# Create Tv Shows Url List

tv_shows_url_list = []
urls = soup.findAll('a' , {'class':"ipc-lockup-overlay ipc-focusable"})
for i in range(250):
  tv_shows_url_list.append("https://m.imdb.com/" + urls[i]["href"])
print(tv_shows_url_list)



['https://m.imdb.com//title/tt0903747/?ref_=chttvtp_i_1', 'https://m.imdb.com//title/tt5491994/?ref_=chttvtp_i_2', 'https://m.imdb.com//title/tt0795176/?ref_=chttvtp_i_3', 'https://m.imdb.com//title/tt0185906/?ref_=chttvtp_i_4', 'https://m.imdb.com//title/tt7366338/?ref_=chttvtp_i_5', 'https://m.imdb.com//title/tt0306414/?ref_=chttvtp_i_6', 'https://m.imdb.com//title/tt0417299/?ref_=chttvtp_i_7', 'https://m.imdb.com//title/tt6769208/?ref_=chttvtp_i_8', 'https://m.imdb.com//title/tt0141842/?ref_=chttvtp_i_9', 'https://m.imdb.com//title/tt2395695/?ref_=chttvtp_i_10', 'https://m.imdb.com//title/tt0081846/?ref_=chttvtp_i_11', 'https://m.imdb.com//title/tt9253866/?ref_=chttvtp_i_12', 'https://m.imdb.com//title/tt0944947/?ref_=chttvtp_i_13', 'https://m.imdb.com//title/tt0071075/?ref_=chttvtp_i_14', 'https://m.imdb.com//title/tt7678620/?ref_=chttvtp_i_15', 'https://m.imdb.com//title/tt1355642/?ref_=chttvtp_i_16', 'https://m.imdb.com//title/tt2861424/?ref_=chttvtp_i_17', 'https://m.imdb.com//t

<ipython-input-3-27e77b59d381>:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  rating_list.append(rating.find_next(text=True).strip())


In [4]:
Movie_image = []
Genre_list = []
for url in tv_shows_url_list:
  time.sleep(1)
  headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
  page = requests.get(url, headers = headers)
  soup = BeautifulSoup(page.text , 'html.parser')
  print(url)

  # Movie Image
  image = soup.find('a' , {'class':"ipc-lockup-overlay ipc-focusable"})
  Movie_image.append("https://m.imdb.com/" + image["href"])

  # Genre
  genre_batch = soup.findAll('a' , {'class':"ipc-chip ipc-chip--on-baseAlt"})
  temp = []
  for genre in genre_batch:
    temp.append(genre.text)
  print(temp)
  Genre_list.append(temp)

  # Director and Stars
  cast_div = soup.find('div', {'class':'sc-b7c53eda-2 iOESUA'})
  cast_u_list = cast_div.find('ul' , {'class':"ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"})
  if cast_u_list: #Check for the div the holds the directors and stars names
    cast_list = cast_u_list.findAll('li', {'class':'ipc-metadata-list__item ipc-metadata-list-item--link'})

    if len(cast_list) < 1:#some shows has different class name for the list that holds the directors and stars names
      cast_list = cast_u_list.findAll('li', {'ipc-metadata-list__item'})


    if len(cast_list) > 1: #check if there is a director
      temp = []
      directors_list = cast_list[0].findAll('li',{'class':'ipc-inline-list__item'})
      for director in directors_list: #loop over the directors names
        link = director.find('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        temp.append(link.find_next(text = True))
      Director_list.append(temp)
      temp = []
      stars_div = cast_list[1].find('div', {'class':'ipc-metadata-list-item__content-container'})
      stars_list2 = stars_div.findAll('li',{'class':'ipc-inline-list__item'})
      for star in stars_list2: #loop over the stars names
        link = star.find('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        temp.append(link.find_next(text = True))
      Stars_list.append(temp)
    else: # Director not found
      Director_list.append("Na")
      stars_div = cast_list[0].find('div', {'class':'ipc-metadata-list-item__content-container'})
      stars_list2 = stars_div.findAll('li',{'class':'ipc-inline-list__item'})
      temp = []
      for star in stars_list2:
        link = star.find('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        temp.append(link.find_next(text = True))
      Stars_list.append(temp)
  else: # Neither the Director nor the stars are found
    Director_list.append("Na")
    Stars_list.append("Na")

print(Stars_list)
print(Director_list)

https://m.imdb.com//title/tt0903747/?ref_=chttvtp_i_1
['Crime', 'Drama', 'Thriller']
2


<ipython-input-4-2d7571f51cf6>:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  temp.append(link.find_next(text = True))
<ipython-input-4-2d7571f51cf6>:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  temp.append(link.find_next(text = True))


https://m.imdb.com//title/tt5491994/?ref_=chttvtp_i_2
['Documentary']
1


<ipython-input-4-2d7571f51cf6>:52: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  temp.append(link.find_next(text = True))


https://m.imdb.com//title/tt0795176/?ref_=chttvtp_i_3
['Documentary', 'Family']
1
https://m.imdb.com//title/tt0185906/?ref_=chttvtp_i_4
['Drama', 'History', 'War']
1
https://m.imdb.com//title/tt7366338/?ref_=chttvtp_i_5
['Drama', 'History', 'Thriller']
2
https://m.imdb.com//title/tt0306414/?ref_=chttvtp_i_6
['Crime', 'Drama', 'Thriller']
2
https://m.imdb.com//title/tt0417299/?ref_=chttvtp_i_7
['Animation', 'Action', 'Adventure']
2
https://m.imdb.com//title/tt6769208/?ref_=chttvtp_i_8
['Documentary']
1
https://m.imdb.com//title/tt0141842/?ref_=chttvtp_i_9
['Crime', 'Drama']
2
https://m.imdb.com//title/tt2395695/?ref_=chttvtp_i_10
['Documentary']
1
https://m.imdb.com//title/tt0081846/?ref_=chttvtp_i_11
['Documentary']
2
https://m.imdb.com//title/tt9253866/?ref_=chttvtp_i_12
['Documentary']
1
https://m.imdb.com//title/tt0944947/?ref_=chttvtp_i_13
['Action', 'Adventure', 'Drama']
2
https://m.imdb.com//title/tt0071075/?ref_=chttvtp_i_14
['Documentary', 'History', 'War']
1
https://m.imdb.com

In [5]:
Final_df = pd.DataFrame({'Film Title' : Film_title_list,
                         'IMDb Rating' : rating_list,
                         'Release Date': Release_year_list,
                         'Genre': Genre_list,
                         'Director': Director_list,
                         'Movie Image': Movie_image,
                         'Stars': Stars_list})
Final_df

,Film Title,IMDb Rating,Release Date,Genre,Director,Movie Image,Stars
0,1. Breaking Bad,9.5,2008–2013,"[Crime, Drama, Thriller]",[Vince Gilligan],https://m.imdb.com//title/tt0903747/mediaviewe...,"[Bryan Cranston, Aaron Paul, Anna Gunn]"
1,2. Planet Earth II,9.5,2016,[Documentary],Na,https://m.imdb.com//title/tt5491994/mediaviewe...,"[David Attenborough, Chadden Hunter, Gordon Bu..."
2,3. Planet Earth,9.4,2006,"[Documentary, Family]",Na,https://m.imdb.com//title/tt0795176/mediaviewe...,"[Sigourney Weaver, David Attenborough, Nikolay..."
3,4. Band of Brothers,9.4,2001,"[Drama, History, War]",Na,https://m.imdb.com//title/tt0185906/mediaviewe...,"[Scott Grimes, Damian Lewis, Ron Livingston]"
4,5. Chernobyl,9.3,2019,"[Drama, History, Thriller]",[Craig Mazin],https://m.imdb.com//title/tt7366338/mediaviewe...,"[Jessie Buckley, Jared Harris, Stellan Skarsgård]"
...,...,...,...,...,...,...,...
245,246. Gintama,8.7,2005–2021,"[Animation, Action, Comedy]",Na,https://m.imdb.com//title/tt0988818/mediaviewe...,"[Tomokazu Sugita, Daisuke Sakaguchi, Rie Kugim..."
246,247. Alchemy of Souls,8.7,2022–,"[Action, Drama, Fantasy]",Na,https://m.imdb.com//title/tt20859920/mediaview...,"[Lee Jae-wook, Hwang Min-hyun, Yoo Joon-sang]"
247,248. Tear Along the Dotted Line,8.6,2021,"[Animation, Comedy, Drama]",[Zerocalcare],https://m.imdb.com//title/tt15614372/mediaview...,"[Zerocalcare, Valerio Mastandrea, Wayne Forester]"
248,249. Chef's Table,8.5,2015–2019,"[Documentary, Biography]",Na,https://m.imdb.com//title/tt4295140/mediaviewe...,"[Massimo Bottura, Ruth Reichl, Francis Mallmann]"


In [6]:
Final_df.to_excel('Data.xlsx')